# 🛒 Market Basket Analysis (Sepet Analizi)

Bu notebook, satış verilerini kullanarak **"Müşteriler hangi ürünleri birlikte alıyor?"** sorusuna yanıt arar.
Kullanılan Teknik: **Apriori Algoritması** & **Association Rules**.

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Veri Yükleme
df = pd.read_csv('../data/mock_sales_data.csv')
print(df.shape)
df.head()

## 1. Veri Hazırlığı (Transaction Matrix)
Satırların fiş/sepet (Store + Date), sütunların ürün olduğu bir matris oluşturuyoruz.

In [ ]:
# Basit olması için 'Store + Date' kombinasyonunu bir sepet (Basket) olarak kabul edelim.
df['basket_id'] = df['store_id'] + '_' + df['date']

# Pivot Table: Sepet x Ürün
basket = (df.groupby(['basket_id', 'category'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('basket_id'))

# Binary Encode
def encode_units(x):
    return 1 if x >= 1 else 0

basket_sets = basket.applymap(encode_units)
basket_sets.head()

## 2. Apriori ile Kuralları Çıkarma
Minimum Support ve Lift değerlerine göre ilişkileri bulalım.

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

rules = rules.sort_values(['lift'], ascending=False)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

## 3. Yorumlar
- **Lift > 1:** İki ürünün birlikte satılma ihtimali tesadüften yüksektir. (Birbirini pozitif etkiliyor)
- **Confidence:** X alındığında Y'nin alınma olasılığı.